In [1]:
import pandas as pd
from matplotlib import pyplot as plt

In [16]:
data_dir = 'data/'
gpu_info = pd.read_csv(data_dir + 'experiments.csv')
gpu_info['gpu-utilization'] = gpu_info['gpu-utilization'].astype(object)
gpu_info['gpu-memory'] = gpu_info['gpu-memory'].astype(object)
gpu_info['gpu-power-usage'] = gpu_info['gpu-power-usage'].astype(object)
gpu_info['gpu-temperature'] = gpu_info['gpu-temperature'].astype(object)

In [17]:
grafana_gpu_power_usage = pd.read_csv(data_dir + 'grafana_gpu_power_usage.csv', sep=';')

In [18]:
grafana_gpu_utilization = pd.read_csv(data_dir + 'grafana_gpu_utilization.csv', sep=';')

In [19]:
grafana_gpu_memory = pd.read_csv(data_dir + 'grafana_gpu_memory.csv', sep=';')

In [20]:
grafana_gpu_temperature = pd.read_csv(data_dir + 'grafana_gpu_temperature.csv', sep=';')

In [21]:
import pytz
import datetime
from datetime import timezone
from numpy import trapz
import numpy as np

def get_grafana_info(gpu_info, grafana_df, tolerance_min=15):
    grafana_df['Time'] = pd.to_datetime(grafana_df['Time'], format='%Y-%m-%dT%H:%M:%S')
    grafana_df['Time'] = grafana_df['Time'].dt.tz_convert(tz='Etc/GMT-2')

    start_time = pd.date_range(gpu_info['start-time'], periods=1) + pd.Timedelta(minutes=-tolerance_min)
    end_time = pd.date_range(gpu_info['end-time'], periods=1) + pd.Timedelta(minutes=tolerance_min)

    new_start = pd.date_range(start_time[0].replace(tzinfo=timezone.utc), periods=1)
    new_end = pd.date_range(end_time[0].replace(tzinfo=timezone.utc), periods=1)
    
    res_df = grafana_df.loc[(grafana_df['Series'] == gpu_info['nvidia-plugins'] + '-gpu0') & \
                            (grafana_df['Time'] > new_start[0]) & \
                            (grafana_df['Time'] <= new_end[0])]
    
    values = res_df['Value'].values
    print('\t' + str(len(values)))
    
    #plt.plot(range(0, len(values)), values)
    #plt.show()
    
    area = trapz(values, range(0, len(values)), 1)
    print("\tarea =", area)
    
    return values

In [22]:
def print_grafana_info(values, label='info'):
    if all(v == 0 for v in values):
        print('\tERROR')
    else:
        print('\t' + label + ' = ' + str(max(values)))

for index, row in gpu_info.iterrows():
    print(row['model'], row['batch-size'], row['nvidia-plugins'])
    tolerance_min = 3
    
    gpu_utilization = get_grafana_info(row, grafana_gpu_utilization, tolerance_min)
    print_grafana_info(gpu_utilization, 'gpu_utilization')
    gpu_info.at[index, 'gpu-utilization'] = gpu_utilization
    
    gpu_memory = get_grafana_info(row, grafana_gpu_memory)
    print_grafana_info(gpu_memory, 'gpu_memory')
    gpu_info.at[index, 'gpu-memory'] = gpu_memory
        
    gpu_power_usage = get_grafana_info(row, grafana_gpu_power_usage, tolerance_min)
    print_grafana_info(gpu_power_usage, 'gpu_power_usage')
    gpu_info.at[index, 'gpu-power-usage'] = gpu_power_usage
    
    gpu_temperature = get_grafana_info(row, grafana_gpu_temperature, tolerance_min)
    print_grafana_info(gpu_temperature, 'gpu_temperature')
    gpu_info.at[index, 'gpu-temperature'] = gpu_temperature
        
    print()

gpu_info.to_csv('results/experiments-with-grafana.csv')

alexnet 1024 nvidia-gpu-device-plugin-k6jh2
	134
	area = 122.0
	gpu_utilization = 49
	134
	area = 3263.134410421248
	gpu_memory = 95.38745174635885
	134
	area = 4250090.5
	gpu_power_usage = 54839
	134
	area = 4392.0
	gpu_temperature = 38

alexnet 128 nvidia-gpu-device-plugin-lbhl2
	135
	area = 0.0
	ERROR
	135
	area = 3452.8834914873646
	gpu_memory = 95.38129988772856
	135
	area = 3933168.0
	gpu_power_usage = 35978
	135
	area = 4097.5
	gpu_temperature = 32

alexnet 2048 nvidia-gpu-device-plugin-mw5pt
	136
	area = 260.0
	gpu_utilization = 100
	136
	area = 3456.892196367327
	gpu_memory = 95.39975546361946
	136
	area = 5512866.0
	gpu_power_usage = 291186
	136
	area = 4449.0
	gpu_temperature = 41

alexnet 256 nvidia-gpu-device-plugin-j8nl9
	133
	area = 18.0
	gpu_utilization = 9
	133
	area = 3059.8022177450357
	gpu_memory = 95.38129988772856
	133
	area = 4047415.0
	gpu_power_usage = 58098
	133
	area = 4778.0
	gpu_temperature = 39

alexnet 32 nvidia-gpu-device-plugin-j4xh9
	135
	area = 6.0
	g